In [1]:
import os
import json
import gc
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer
from typing import Dict, List, Set, Tuple, NamedTuple, Callable
import scipy
import scml
from scml import pandasx as pdx
from daigt.preprocess import en as pen
tim = scml.Timer()
tim.start()
os.environ["TOKENIZERS_PARALLELISM"] = "false"
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()
info = np.iinfo(np.int16)
print(f"int16, min={info.min}, max={info.max}")

int16, min=-32768, max=32767


In [2]:
min_len, max_len = 773, 32_767  # P01 cutoff

In [3]:
#tokenizer = AutoTokenizer.from_pretrained("huggingface/microsoft/deberta-v3-base", is_fast=True)
#print(f"{repr(tokenizer)}\nmodel_input_names={tokenizer.model_input_names}")

# Combine data sources

In [4]:
edf = pd.read_csv("input/train_essays.csv", low_memory=False)
edf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1378 entries, 0 to 1377
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         1378 non-null   object
 1   prompt_id  1378 non-null   int64 
 2   text       1378 non-null   object
 3   generated  1378 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 43.2+ KB


In [5]:
pdf = pd.read_csv("input/train_prompts.csv", low_memory=False)
pdf = pdf.set_index("prompt_id")
pdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, 0 to 1
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   prompt_name   2 non-null      object
 1   instructions  2 non-null      object
 2   source_text   2 non-null      object
dtypes: object(3)
memory usage: 64.0+ bytes


In [6]:
df = pd.read_csv("input/train_drcat_02.csv", low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39785 entries, 0 to 39784
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   essay_id  39785 non-null  object
 1   text      39785 non-null  object
 2   label     39785 non-null  int64 
 3   source    39785 non-null  object
 4   prompt    9490 non-null   object
 5   fold      39785 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1.8+ MB


In [7]:
cols = ["essay_id", "text", "label", "source", "prompt"]
df = df[cols]
df = df.rename(columns={"essay_id": "id", "label": "generated"})
df["prompt"] = df["prompt"].fillna("na")

In [8]:
def get_prompt(row)->str:
    p=pdf.loc[row["prompt_id"]]
    name=p["prompt_name"]
    inst=p["instructions"]
    return f"{name} {inst}"
    

edf["prompt"] = edf.progress_apply(get_prompt, axis=1)
edf["source"] = "train_essays"
df = pd.concat([edf, df], ignore_index=True)
cols = ["id", "generated", "source", "prompt", "text"]
df = df[cols].copy()
cols = ["generated"]
df[cols] = df[cols].astype(np.int8)
df.info()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1378/1378 [00:00<00:00, 75769.21it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41163 entries, 0 to 41162
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         41163 non-null  object
 1   generated  41163 non-null  int8  
 2   source     41163 non-null  object
 3   prompt     41163 non-null  object
 4   text       41163 non-null  object
dtypes: int8(1), object(4)
memory usage: 1.3+ MB


In [9]:
more = len(df)
df = df.drop_duplicates(["text"], ignore_index=True)
print(f"{more - len(df)} rows dropped: duplicates")

1429 rows dropped: duplicates


# Preprocess Text

In [10]:
bsc = pen.BasicPreprocessor()
bow = pen.BowPreprocessor()

def preprocess_bsc(col) -> Callable:
    def fn(row) -> str:
        return bsc(row[col])
    
    return fn

def preprocess_bow(col) -> Callable:
    def fn(row) -> str:
        return bow(row[col], drop_stopword=False)
    
    return fn

In [11]:
cols = ["text", "prompt"]
for col in cols:
    print(col)
    new_col = f"{col}_bsc" 
    df[new_col] = df.progress_apply(preprocess_bsc(col), axis=1)
    new_col = f"{col}_bow" 
    df[new_col] = df.progress_apply(preprocess_bow(col), axis=1)
    col1, col2 = new_col, f"{col}_bow_len"
    df[col2] = df[col1].str.len()
    df[col2] = df[col2].astype(np.int16)
    #x = tokenizer(df[new_col].tolist(), truncation=False, add_special_tokens=False)
    #len_col = f"{new_col}_len" 
    #df[len_col] = [len(s) for s in x["input_ids"]]
    #df[len_col] = df[len_col].astype(np.int16) 

text


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39734/39734 [16:42<00:00, 39.62it/s]


prompt


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39734/39734 [00:43<00:00, 921.40it/s]


# Filer Invalid Data

In [12]:
more = len(df)
df = df.drop_duplicates(["text_bow"], ignore_index=True)
print(f"{more - len(df)} rows dropped: duplicates")

219 rows dropped: duplicates


In [13]:
more = len(df)
df = df[(df["text_bow_len"]>=min_len) & (df["text_bow_len"]<=max_len)].copy()
print(f"{more - len(df)} rows dropped: duplicates")

0 rows dropped: duplicates


# Review Data

In [14]:
df = df.reset_index(drop=True) 
df = df.drop(columns=["id"])
df["essay_id"] = df.index
df["essay_id"] = df["essay_id"].astype(np.int32)
cols = list(df.columns)
cols = [cols[-1]] + cols[:-1]
df = df[cols]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39515 entries, 0 to 39514
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   essay_id        39515 non-null  int64 
 1   generated       39515 non-null  int8  
 2   source          39515 non-null  object
 3   prompt          39515 non-null  object
 4   text            39515 non-null  object
 5   text_bsc        39515 non-null  object
 6   text_bow        39515 non-null  object
 7   text_bow_len    39515 non-null  int32 
 8   prompt_bsc      39515 non-null  object
 9   prompt_bow      39515 non-null  object
 10  prompt_bow_len  39515 non-null  int32 
dtypes: int32(2), int64(1), int8(1), object(7)
memory usage: 2.8+ MB


In [15]:
df.describe(percentiles=percentiles)

,essay_id,generated,text_bow_len,prompt_bow_len
count,39515.00000,39515.000000,39515.000000,39515.000000
mean,19757.00000,0.252891,2195.112818,87.917854
std,11407.14228,0.434675,1006.480680,161.061500
min,0.00000,0.000000,234.000000,2.000000
1%,395.14000,0.000000,773.000000,2.000000
5%,1975.70000,0.000000,951.000000,2.000000
10%,3951.40000,0.000000,1100.000000,2.000000
20%,7902.80000,0.000000,1359.000000,2.000000
30%,11854.20000,0.000000,1582.000000,2.000000
40%,15805.60000,0.000000,1792.000000,2.000000


In [16]:
cols=["generated"]
pdx.value_counts(df[cols])

,count,percent
generated,,
0,29522,0.747109
1,9993,0.252891


In [17]:
cols=["prompt_bow"]
pdx.value_counts(df[cols]).head(50)

,count,percent
prompt_bow,,
na,28769,0.728053
car free cities write an explanatory essay to inform fellow citizens about the advantages of limiting car usage your essay must be based on ideas and information that can be found in the passage set manage your time carefully so that you can read the passages plan your response write your response and revise and edit your response be sure to use evidence from multiple sources and avoid overly relying on one source your response should be in the form of a multiparagraph essay write your essay in the space provided,708,0.017917
does the electoral college work write a letter to your state senator in which you argue in favor of keeping the electoral college or changing to election by popular vote for the president of the united states use the information from the texts in your essay manage your time carefully so that you can read the passages plan your response write your response and revise and edit your response be sure to include a claim address counterclaims use evidence from multiple sources and avoid overly relying on one source your response should be in the form of a multiparagraph essay write your response in the space provided,670,0.016956
the legalization of marijuana have been a controversial topic in recent years write an essay arguing for or against the legalization of marijuana providing reasons and examples to support your position,77,0.001949
some schools offer distance learning as an option for students to attend classes from home by way of online or video conferencing do you think students would benefit from being able to attend classes from home take a position on this issue support your response with reasons and examples,77,0.001949
the role of libraries in the digital age have been a topic of discussion write an essay arguing for or against the continued importance of libraries providing reasons and examples to support your position,75,0.001898
write an explanatory essay to inform fellow citizens about the advantages of limiting car usage your essay must be based on ideas and information that can be found in the passage set manage your time carefully so that you can read the passages plan your response write your response and revise and edit your response be sure to use evidence from multiple sources and avoid overly relying on one source your response should be in the form of a multiparagraph essay write your essay in the space provided,74,0.001873
the role of zoos in conservation and education have been a topic of debate argue for or against the continued operation of zoos providing reasons and examples to justify your position,72,0.001822
the use of single use plastic products have become a controversial topic due to its impacts on the environment write an essay arguing for or against the ban of single use plastics providing reasons and examples to support your position,71,0.001797


In [18]:
cols=["source"]
pdx.value_counts(df[cols])

,count,percent
source,,
persuade_corpus,25848,0.654131
chat_gpt_moth,2421,0.061268
llama2_chat,2421,0.061268
llammistral7binstruct,2421,0.061268
original_moth,2299,0.058180
train_essays,1378,0.034873
llama_70b_v1,1172,0.029660
falcon_180b_v1,1055,0.026699
radek_500,500,0.012653


In [19]:
df.sample(5).head()

,essay_id,generated,source,prompt,text,text_bsc,text_bow,text_bow_len,prompt_bsc,prompt_bow,prompt_bow_len
31613,31613,1,llammistral7binstruct,na,"Failure is an inevitable part of life that can have a significant impact on people's ability to achieve success. While failure can be discouraging and demotivating, it can also be beneficial in showing maturity and helping with enthusiasm.\n\nFirstly, failure can be a valuable learning experience that helps individuals develop resilience and perseverance. When people fail, they are forced to re-evaluate their approach and find new ways to solve problems. This process can help them become more resourceful and adaptable, which are essential skills for success. For example, Thomas Edison failed thousands of times before finally inventing the light bulb. However, he used each failure as an opportunity to learn and improve, eventually leading to his success.\n\nSecondly, failure can help individuals develop a growth mindset, which is the belief that abilities and intelligence can be developed through hard work and dedication. When people fail, they are forced to acknowledge their weaknesses and work towards improving them. This process can help them become more self-aware and confident, which are crucial for achieving success. For example, Michael Jordan missed over 9,000 shots in his career, but he used this failure as motivation to become an even better basketball player.\n\nFinally, failure can be a source of motivation and enthusiasm. When people fail, they are often driven to work harder and smarter to avoid future failures. This increased effort and focus can lead to greater success. For example, J.K. Rowling faced numerous rejections before finally finding a publisher for her Harry Potter series. However, she used this failure as motivation to continue writing and eventually achieved tremendous success.\n\nIn conclusion, failure can play a significant role in people's lives in achieving success. While failure can be discouraging, it can also be a valuable learning experience that helps individuals develop resilience and perseverance. Failure can also help individuals develop a growth mindset, which is essential for achieving success. Finally, failure can be a source of motivation and enthusiasm that drives individuals to work harder and smarter to avoid future failures.","Failure is an inevitable part of life that can have a significant impact on people's ability to achieve success. While failure can be discouraging and demotivating, it can also be beneficial in showing maturity and helping with enthusiasm. Firstly, failure can be a valuable learning experience that helps individuals develop resilience and perseverance. When people fail, they are forced to re-evaluate their approach and find new ways to solve problems. This process can help them become more resourceful and adaptable, which are essential skills for success. For example, Thomas Edison failed thousands of times before finally inventing the light bulb. However, he used each failure as an opportunity to learn and improve, eventually leading to his success. Secondly, failure can help individuals develop a growth mindset, which is the belief that abilities and intelligence can be developed through hard work and dedication. When people fail, they are forced to acknowledge their weaknesses and work towards improving them. This process can help them become more self-aware and confident, which are crucial for achieving success. For example, Michael Jordan missed over 9,000 shots in his career, but he used this failure as motivation to become an even better basketball player. Finally, failure can be a source of motivation and enthusiasm. When people fail, they are often driven to work harder and smarter to avoid future failures. This increased effort and focus can lead to greater success. For example, J.K. Rowling faced numerous rejections before finally finding a publisher for her Harry Potter series. However, she used this 

In [20]:
%%time
df.to_parquet(f"output/preprocess.parquet", index=False)
assert df.notna().all(axis=None)

CPU times: user 545 ms, sys: 48.7 ms, total: 594 ms
Wall time: 595 ms


In [21]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 0:17:39.943724
